In [1]:
import pandas as pd
import numpy as np

In [2]:
offensive = pd.read_excel("/content/drive/MyDrive/pittsburgh-annotation-son-20240613/hate_speech_isik_llm_model_positive_preds_evaluation_offensive_hate_annotation_busra_kadir_merged_20240603_2nd.xlsx")

In [3]:
offensive.columns

Index(['Annotator', '#', 'text', 'Dis\nNeutral', 'Offense \nNeutral',
       'dis_O_ Relevant', 'Offense \nRelevant', 'Dis_o_Physical harm',
       'Offense Physical Harm Threat', 'dis_o_insult_prof',
       'Offense Insult profanity', 'dis_o_insult_demean',
       'Offense Insult\n(Demean)', 'dis_o_ridicule',
       'Offense Ridicule Mockery', 'dis_o_other', 'Offense \nOther', 'dis_HS',
       'Hate \nSpeech', 'dis_H_eth', 'Hate \nEthnicity', 'dis_H_rel',
       'Hate  \nReligion', 'Dis_H_Gen', 'Hate \nGender', 'Dis_H_other',
       'Hate \nOther', 'comment'],
      dtype='object')

In [4]:
offensive.rename(columns = {'Hate \nSpeech':"hate-speech"},inplace=True)

In [5]:
offensive["hate-speech"]

0          1.0
1          1.0
2          NaN
3          1.0
4          1.0
         ...  
3226       NaN
3227       NaN
3228       NaN
3229    1303.0
3230     182.0
Name: hate-speech, Length: 3231, dtype: float64

In [6]:
offensive["dis_HS"]

0              NaN
1              NaN
2         2.000000
3              NaN
4              NaN
           ...    
3226    412.000000
3227    279.000000
3228    147.670251
3229           NaN
3230           NaN
Name: dis_HS, Length: 3231, dtype: float64

In [7]:
import pandas as pd

def resolve_hate_speech(group):
    busra_value = group[group['Annotator'] == 'busra']['hate-speech'].values
    kadir_value = group[group['Annotator'] == 'kadir']['hate-speech'].values
    final_value = group[group['Annotator'] == 'final']['hate-speech'].values

    if len(busra_value) > 0 and len(kadir_value) > 0:
        busra_value = busra_value[0]
        kadir_value = kadir_value[0]
        if busra_value == kadir_value:
            return busra_value
        elif len(final_value) > 0:
            return final_value[0]
        else:
            return None
    else:
        return None

grouped = offensive.groupby('text')
resolved_values = grouped.apply(resolve_hate_speech)
offensive['Resolved_Hate_Speech'] = offensive['text'].map(resolved_values).astype('Int64')
offensive_final = offensive.drop_duplicates(subset='text', keep='first')
offensive_final = offensive_final.reset_index(drop=True)
offensive_final = offensive_final[['text', 'Resolved_Hate_Speech']]
offensive_final["Resolved_Hate_Speech"].astype(str)
print(offensive_final)


                                                   text  Resolved_Hate_Speech
0     @meraldanis @Alone_36_ Sen o kadar vatansevers...                     1
1     @meraldanis Erzurum'da kaza geçirmiş. Ne yazık...                     1
2     @gulderenvarli Emekçi kürt halkın verdiği verg...                     1
3     @meraldanis Bunlari bebek katili savunan diyor...                     1
4     @meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...                     1
...                                                 ...                   ...
1076                                              agree                  <NA>
1077                                           disagree                  <NA>
1078                                          agree/dis                  <NA>
1079                                              total                  <NA>
1080                                           subtotal                  <NA>

[1081 rows x 2 columns]


In [8]:
offensive_final["Resolved_Hate_Speech"].value_counts()

Resolved_Hate_Speech
1    612
0    463
Name: count, dtype: Int64

In [9]:
offensive_final.rename(columns={"Resolved_Hate_Speech":"labels"},inplace=True)

In [10]:
offensive_final

,text,labels
0,@meraldanis @Alone_36_ Sen o kadar vatansevers...,1
1,@meraldanis Erzurum'da kaza geçirmiş. Ne yazık...,1
2,@gulderenvarli Emekçi kürt halkın verdiği verg...,1
3,@meraldanis Bunlari bebek katili savunan diyor...,1
4,@meraldanis Dağdaki ler aç mı ?kaldı kurşunu b...,1
...,...,...
1076,agree,<NA>
1077,disagree,<NA>
1078,agree/dis,<NA>
1079,total,<NA>


In [11]:

import pandas as pd

# Assuming 'offensive_final' is already loaded
# Replace string '' with actual pd.NA
offensive_final['labels'] = offensive_final['labels'].replace('', pd.NA)

# Convert labels to nullable integer type
offensive_final['labels'] = offensive_final['labels'].astype('Int64')

# Drop rows where 'labels' is pd.NA
offensive_final = offensive_final.dropna(subset=['labels'])

# Verify the cleanup
print(offensive_final['labels'].value_counts())


labels
1    612
0    463
Name: count, dtype: Int64


In [12]:
offensive_final["labels"].value_counts()

labels
1    612
0    463
Name: count, dtype: Int64

In [13]:
offensive_final.dtypes

text      object
labels     Int64
dtype: object

In [14]:
offensive_final['labels'] = offensive_final['labels'].astype(str)

<ipython-input-14-9220c4dc9513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offensive_final['labels'] = offensive_final['labels'].astype(str)


In [15]:
offensive_final.dtypes

text      object
labels    object
dtype: object

In [ ]:
offensive_final["labels"].value_counts()

In [20]:
offensive_final.dtypes

text      object
labels    object
dtype: object

In [ ]:
offensive_final = offensive_final.reset_index(drop=True)
hate = hate.reset_index(drop=True)

# Concatenate the DataFrames along the columns
offensive_final_concat = pd.concat([offensive_final, hate], axis=0)

# Verify the result
print(offensive_final_concat.head())

In [ ]:
offensive_final = offensive_final_concat

In [ ]:
offensive_final["labels"].value_counts()

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
train_df, test_df = train_test_split(offensive_final, test_size=0.2, random_state=42)

In [19]:
train_df.dtypes

text      object
labels    object
dtype: object

In [ ]:
#!pip install peft urlextract ftfy

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/pittsburgh-annotation/hate-speech-fırat-annotated.xlsx")

In [ ]:
df["Hate Speech"] = df["Hate Speech"].fillna("0")
df['Hate Speech'] = df['Hate Speech'].astype(int)

In [ ]:
df['Hate Speech'].value_counts()

In [ ]:
df.rename(columns={"Hate Speech":"label"},inplace=True)
df.reset_index(drop=True)

In [ ]:
df["label"].value_counts()

In [ ]:
df = df[["text","label"]]

In [ ]:
hate = df[df["label"]==1]

In [ ]:
hate.rename(columns={"label":"labels"},inplace=True)

In [ ]:
hate["labels"] = hate["labels"].astype(str)

In [ ]:
hate

In [ ]:
offensive_final["labels"].value_counts()

In [ ]:
offensive_final["labels"] = offensive_final["labels"].astype(str)

In [ ]:
offensive_final = pd.merge(offensive_final , hate , on="labels")

In [ ]:
offensive_final["labels"].value_counts()

In [21]:
train_df.to_json("/content/drive/MyDrive/pittsburgh-annotation/hate_binary_train.json",orient="records",lines=True)

In [22]:
offensive_final.to_json("/content/drive/MyDrive/pittsburgh-annotation/hate_binary.json",orient="records",lines=True)

In [23]:
test_df.to_json("/content/drive/MyDrive/pittsburgh-annotation/hate_binary_test.json",orient="records",lines=True)

In [24]:
from sklearn.metrics import precision_recall_fscore_support, matthews_corrcoef
import torch
from transformers import AutoModel, AutoTokenizer, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
import numpy as np
import time
import random
from tqdm import tqdm
import json
import os
from sklearn.utils import resample
import pandas as pd

class TransformersData(torch.utils.data.Dataset):
    def __init__(self, examples, label_map, tokenizer, binary=False, max_seq_length=512, has_token_type_ids=False, with_label=True):
        self.examples = examples
        self.label_map = label_map
        self.binary = binary

        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.with_label = with_label
        self.has_token_type_ids = has_token_type_ids

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        encoded_input = self.tokenizer(ex[0], padding="max_length", truncation=True, max_length=self.max_seq_length)
        input_ids = torch.tensor(encoded_input["input_ids"], dtype=torch.long)
        input_mask = torch.tensor(encoded_input["attention_mask"], dtype=torch.long)
        if self.has_token_type_ids:
            token_type_ids = torch.tensor(encoded_input["token_type_ids"], dtype=torch.long)

        if self.with_label:
            if self.binary:
                label_ids = torch.FloatTensor([self.label_map[int(ex[1])]])
            else:
                label_ids = torch.tensor(self.label_map[int(ex[1])], dtype=torch.long)

            if self.has_token_type_ids:
                return input_ids, input_mask, token_type_ids, label_ids
            else:
                return input_ids, input_mask, label_ids

        if self.has_token_type_ids:
            return input_ids, input_mask, token_type_ids
        else:
            return input_ids, input_mask

def get_examples(filename, with_label=True):
    with open(filename, "r", encoding="utf-8") as f:
        lines = f.read().splitlines()

    examples = []
    for (i, line) in enumerate(lines):
        line = json.loads(line)
        text = str(line["text"])
        if with_label:
            label = int(line["labels"])
            examples.append([text, label])
        else:
            examples.append([text])

    return examples

# INPUTS
pretrained_transformers_model = "dbmdz/bert-base-turkish-128k-cased" # Pretrained model
seed = 42
max_seq_length = 512 # max length of a document (in tokens)
batch_size = 8
dev_ratio = 0.1

# MUST SET THESE VALUES
repo_path = "/content/drive/MyDrive/pittsburgh-annotation"
train_filename = repo_path + "/hate_binary_train.json"
test_filename = repo_path + "/hate_binary_test.json"
only_test = False # Only perform testing
predict = False # Predict instead of testing
has_token_type_ids = False

# Define label lists and mappings for binary classification
label_list = [0, 1]  # Directly using integers as labels
label_to_idx = {label: idx for idx, label in enumerate(label_list)}
idx_to_label = {idx: label for idx, label in enumerate(label_list)}

# Load the dataset and perform resampling
offensive_final = pd.read_json("/content/drive/MyDrive/pittsburgh-annotation/hate_binary.json", orient="records", lines=True)
df_majority = offensive_final[offensive_final.labels == 0]
df_minority = offensive_final[offensive_final.labels == 1]

df_minority_upsampled = resample(df_minority,
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # reproducible results

df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Save the upsampled data to train_filename
df_upsampled.to_json(train_filename, orient='records', lines=True)

# SETTINGS
binary = len(label_list) == 2
learning_rate = 2e-5
dev_metric = "f1_macro"
num_epochs = 10
dev_set_splitting = "random" # random, or any filename
use_gpu = True
device_ids = [0] # if not multi-gpu then pass a single number such as [0]
positive_threshold = 0.5 # Outputted probabilities bigger than this number is considered positive in case of binary classifications
return_probabilities = False # whether or not to return probabilities instead of labels when predicting
model_path = "{}_{}_{}_{:.2f}_{}.pt".format(pretrained_transformers_model.replace("/", "_"), max_seq_length, batch_size, dev_ratio, seed)

# optional, used in testing
classifier_path = ""
encoder_path = ""

if not classifier_path:
    classifier_path =  repo_path + "/models/classifier_hate_binary_" + model_path
if not encoder_path:
    encoder_path =  repo_path + "/models/encoder_hate_binary_" + model_path

if return_probabilities:
    from scipy.special import softmax

if use_gpu and torch.cuda.is_available():
    device = torch.device("cuda:%d"%(device_ids[0]))
else:
    device = torch.device("cpu")

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device.type == "cuda":
    torch.cuda.manual_seed_all(seed)

tokenizer = None
criterion = torch.nn.BCEWithLogitsLoss() if binary else torch.nn.CrossEntropyLoss(ignore_index=-1)

def test_model(encoder, classifier, dataloader):
    all_preds = []
    all_label_ids = []
    eval_loss = 0
    nb_eval_steps = 0
    for val_step, batch in enumerate(dataloader):
        if has_token_type_ids:
            input_ids, input_mask, token_type_ids, label_ids = tuple(t.to(device) for t in batch)
            embeddings = encoder(input_ids, attention_mask=input_mask, token_type_ids=token_type_ids)[1]
        else:
            input_ids, input_mask, label_ids = tuple(t.to(device) for t in batch)
            embeddings = encoder(input_ids, attention_mask=input_mask)[1]

        with torch.no_grad():
            out = classifier(embeddings)
            tmp_eval_loss = criterion(out, label_ids)

        eval_loss += tmp_eval_loss.mean().item()

        if binary:
            curr_preds = torch.sigmoid(out).detach().cpu().numpy().flatten()
            curr_preds = [int(x >= positive_threshold) for x in curr_preds]
            all_preds += curr_preds
        else:
            out = out.detach().cpu().numpy()
            all_preds += np.argmax(out, axis=1).tolist()

        label_ids = label_ids.to('cpu').numpy().flatten().tolist()
        all_label_ids += label_ids

        nb_eval_steps += 1

    precision, recall, f1, _ = precision_recall_fscore_support(all_label_ids, all_preds, average="macro", labels=list(range(0,len(label_list))))
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(all_label_ids, all_preds, average="micro", labels=list(range(0,len(label_list))))
    mcc = matthews_corrcoef(all_preds, all_label_ids)
    eval_loss /= nb_eval_steps
    result = {"precision_macro": precision,
              "recall_macro": recall,
              "f1_macro": f1,
              "precision_micro": precision_micro,
              "recall_micro": recall_micro,
              "f1_micro": f1_micro,
              "mcc": mcc}

    return result, eval_loss


def model_predict(encoder, classifier, dataloader):
    all_preds = []
    for val_step, batch in enumerate(dataloader):
        if has_token_type_ids:
            input_ids, input_mask, token_type_ids = tuple(t.to(device) for t in batch)
            embeddings = encoder(input_ids, attention_mask=input_mask, token_type_ids=token_type_ids)[1]
        else:
            input_ids, input_mask = tuple(t.to(device) for t in batch)
            embeddings = encoder(input_ids, attention_mask=input_mask)[1]

        with torch.no_grad():
            out = classifier(embeddings)

        if binary:
            curr_preds = torch.sigmoid(out).detach().cpu().numpy().flatten()
            if return_probabilities:
                curr_preds = [round(float(x), 4) for x in curr_preds]
            else:
                curr_preds = [idx_to_label[int(x >= positive_threshold)] for x in curr_preds]
            all_preds += curr_preds

        else:
            out = out.detach().cpu().numpy()
            if return_probabilities:
                curr_preds = [probs for probs in softmax(out, axis=1).tolist()] # a list of lists(of probabilities)
            else:
                curr_preds = [idx_to_label[pred] for pred in np.argmax(out, axis=1).tolist()] # a list of labels
            all_preds += curr_preds

    return all_preds


def build_model(train_examples, dev_examples, pretrained_model, n_epochs=10, curr_model_path="temp.pt"):
    global tokenizer

    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    encoder = AutoModel.from_pretrained(pretrained_model)
    classifier = torch.nn.Linear(encoder.config.hidden_size, 1 if binary else len(label_list))

    train_dataset = TransformersData(train_examples, label_to_idx, tokenizer, binary=binary, max_seq_length=max_seq_length, has_token_type_ids=has_token_type_ids)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    dev_dataset = TransformersData(dev_examples, label_to_idx, tokenizer, binary=binary, max_seq_length=max_seq_length, has_token_type_ids=has_token_type_ids)
    dev_dataloader = DataLoader(dataset=dev_dataset, batch_size=batch_size)

    classifier.to(device)
    if torch.cuda.device_count() > 1 and device.type == "cuda" and len(device_ids) > 1:
        encoder = torch.nn.DataParallel(encoder, device_ids=device_ids)
    encoder.to(device)

    optimizer = torch.optim.AdamW(list(classifier.parameters()) + list(encoder.parameters()), lr=learning_rate)
    num_train_steps = int(len(train_examples) / batch_size * num_epochs)
    # scheduler = get_linear_schedule_with_warmup(optimizer,
    #                                             num_warmup_steps = 0,
    #                                             num_training_steps = num_train_steps)

    best_score = -1e6
    for epoch in range(n_epochs):
        start_time = time.time()
        train_loss = 0
        encoder.train()
        classifier.train()

        print("Starting Epoch %d"%(epoch+1))
        global_step = 0
        train_loss = 0.0
        nb_tr_steps = 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            if has_token_type_ids:
                input_ids, input_mask, token_type_ids, label_ids = tuple(t.to(device) for t in batch)
                embeddings = encoder(input_ids, attention_mask=input_mask, token_type_ids=token_type_ids)[1]
            else:
                input_ids, input_mask, label_ids = tuple(t.to(device) for t in batch)
                embeddings = encoder(input_ids, attention_mask=input_mask)[1]

            out = classifier(embeddings)
            loss = criterion(out, label_ids)

            loss.backward()
            global_step += 1
            nb_tr_steps += 1

            torch.nn.utils.clip_grad_norm_(encoder.parameters(), 1.0)
            torch.nn.utils.clip_grad_norm_(classifier.parameters(), 1.0)
            optimizer.step()
            # scheduler.step()
            encoder.zero_grad()
            classifier.zero_grad()

            train_loss += loss.item()

        train_loss /= nb_tr_steps
        elapsed = time.time() - start_time

        # Validation
        encoder.eval()
        classifier.eval()
        result, val_loss = test_model(encoder, classifier, dev_dataloader)
        result["train_loss"] = train_loss
        result["dev_loss"] = val_loss
        result["elapsed"] = elapsed
        print("***** Epoch " + str(epoch + 1) + " *****")
        for key in sorted(result.keys()):
            print("  %s = %.6f" %(key, result[key]))

        print("Val score: %.6f" %result[dev_metric])

        if result[dev_metric] > best_score:
            print("Saving model!")
            if not os.path.exists(repo_path + "/models"):
                os.makedirs(repo_path + "/models")
            torch.save(classifier.state_dict(), repo_path + "/models/classifier_" + curr_model_path)
            encoder_to_save = encoder.module if hasattr(encoder, 'module') else encoder  # To handle multi gpu
            torch.save(encoder_to_save.state_dict(), repo_path + "/models/encoder_" + curr_model_path)
            best_score = result[dev_metric]

        print("------------------------------------------------------------------------")

    return encoder, classifier

if __name__ == '__main__':
    if dev_set_splitting == "random":
        train_examples = get_examples(train_filename)
        random.shuffle(train_examples)
        dev_split = int(len(train_examples) * dev_ratio)
        dev_examples = train_examples[:dev_split]
        train_examples = train_examples[dev_split:]
    else: # it's a custom filename
        train_examples = get_examples(train_filename)
        random.shuffle(train_examples)
        dev_examples = get_examples(dev_set_splitting)

    if not only_test:
        encoder, classifier = build_model(train_examples, dev_examples, pretrained_transformers_model, n_epochs=num_epochs, curr_model_path=model_path)
        classifier.load_state_dict(torch.load(repo_path + "/models/classifier_" + model_path))
        if torch.cuda.device_count() > 1 and device.type == "cuda" and len(device_ids) > 1:
            encoder.module.load_state_dict(torch.load(repo_path + "/models/encoder_" + model_path))
        else:
            encoder.load_state_dict(torch.load(repo_path + "/models/encoder_" + model_path))
    else:
        tokenizer = AutoTokenizer.from_pretrained(pretrained_transformers_model)
        encoder = AutoModel.from_pretrained(pretrained_transformers_model)
        classifier = torch.nn.Linear(encoder.config.hidden_size, 1 if binary else len(label_list))

        classifier.to(device)
        encoder.to(device)
        classifier.load_state_dict(torch.load(classifier_path, map_location=device))
        encoder.load_state_dict(torch.load(encoder_path, map_location=device))

        if torch.cuda.device_count() > 1 and device.type == "cuda" and len(device_ids) > 1:
            encoder = torch.nn.DataParallel(encoder, device_ids=device_ids)

    encoder.eval()
    classifier.eval()

    if predict:
        test_examples = get_examples(test_filename, with_label=False)
        test_dataset = TransformersData(test_examples, label_to_idx, tokenizer, binary=binary, max_seq_length=max_seq_length, has_token_type_ids=has_token_type_ids, with_label=False)
        test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)

        all_preds = model_predict(encoder, classifier, test_dataloader)
        with open(test_filename, "r", encoding="utf-8") as f:
            test = [json.loads(line) for line in f.read().splitlines()]

        with open(repo_path + "/out.json", "w", encoding="utf-8") as g:
            for i, t in enumerate(test):
                t["prediction"] = all_preds[i]
                g.write(json.dumps(t) + "\n")

    else:
        test_examples = get_examples(test_filename)
        test_dataset = TransformersData(test_examples, label_to_idx, tokenizer, binary=binary, max_seq_length=max_seq_length, has_token_type_ids=has_token_type_ids)
        test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size)

        result, test_loss = test_model(encoder, classifier, test_dataloader)
        result["test_loss"] = test_loss

        print("***** TEST RESULTS *****")
        for key in sorted(result.keys()):
            print("  %s = %.6f" %(key, result[key]))

        print("TEST SCORE: %.6f" %result[dev_metric])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Starting Epoch 1


Iteration: 100%|██████████| 104/104 [00:37<00:00,  2.76it/s]


***** Epoch 1 *****
  dev_loss = 0.577945
  elapsed = 37.651829
  f1_macro = 0.696147
  f1_micro = 0.706522
  mcc = 0.422328
  precision_macro = 0.723162
  precision_micro = 0.706522
  recall_macro = 0.699811
  recall_micro = 0.706522
  train_loss = 0.655365
Val score: 0.696147
Saving model!
------------------------------------------------------------------------
Starting Epoch 2


Iteration: 100%|██████████| 104/104 [00:37<00:00,  2.78it/s]


...

***** Epoch 8 *****
  dev_loss = 1.917705
  elapsed = 37.357524
  f1_macro = 0.751351
  f1_micro = 0.760870
  mcc = 0.540868
  precision_macro = 0.788172
  precision_micro = 0.760870
  recall_macro = 0.753788
  recall_micro = 0.760870
  train_loss = 0.011936
Val score: 0.751351
------------------------------------------------------------------------
Starting Epoch 9


Iteration: 100%|██████████| 104/104 [00:37<00:00,  2.79it/s]


***** Epoch 9 *****
  dev_loss = 1.493962
  elapsed = 37.329214
  f1_macro = 0.790483
  f1_micro = 0.793478
  mcc = 0.590408
  precision_macro = 0.800737
  precision_micro = 0.793478
  recall_macro = 0.789773
  recall_micro = 0.793478
  train_loss = 0.016913
Val score: 0.790483
------------------------------------------------------------------------
Starting Epoch 10


Iteration: 100%|██████████| 104/104 [00:37<00:00,  2.79it/s]


***** Epoch 10 *****
  dev_loss = 2.232972
  elapsed = 37.301705
  f1_macro = 0.726190
  f1_micro = 0.739130
  mcc = 0.501688
  precision_macro = 0.772321
  precision_micro = 0.739130
  recall_macro = 0.731061
  recall_micro = 0.739130
  train_loss = 0.011982
Val score: 0.726190
------------------------------------------------------------------------
***** TEST RESULTS *****
  f1_macro = 0.920869
  f1_micro = 0.920930
  mcc = 0.848506
  precision_macro = 0.922817
  precision_micro = 0.920930
  recall_macro = 0.925693
  recall_micro = 0.920930
  test_loss = 0.550930
TEST SCORE: 0.920869
